In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import xml.etree.ElementTree as ET

def read_anntation(xml_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    bounding_box_list = []

    file_name = root.find('filename').text
    for obj in root.iter('object'):

        class_id = obj.find("name").text

        for box in obj.findall("bndbox"):
            x_min = int(box.find("xmin").text)
            y_min = int(box.find("ymin").text)
            x_max = int(box.find("xmax").text)
            y_max = int(box.find("ymax").text)

        bounding_box = [class_id, x_min, y_min, x_max, y_max]
        bounding_box_list.append(bounding_box)

    return bounding_box_list, file_name

In [3]:
from os import listdir
import cv2
import numpy as np

def read_train_dataset(dir):
    images = []
    annotations = []

    for file in listdir(dir):
        if 'jpg' in file.lower() or 'png' in file.lower():
            images.append(cv2.imread(dir + file, 1))
            annotation_file = file.replace(file.split('.')[-1], 'xml')
            bounding_box_list, file_name = read_anntation(dir + annotation_file)
            annotations.append((bounding_box_list, annotation_file, file_name))

    images = np.array(images)

    return images, annotations

In [4]:
!pip install files

  Created wheel for files: filename=files-1.1.1-py3-none-any.whl size=3669 sha256=ca60ea3c1c1c7e9ab2be34acc1d29c72349290fa6caaacaf5cad04fbedd32d78
  Stored in directory: /root/.cache/pip/wheels/2f/c3/9a/1705fa83402ed949d63209767985876ef4c5c78f8d53415bfb
Successfully built files


In [5]:
pip install pascal-voc-writer

In [15]:
import imgaug as ia
from imgaug import augmenters as iaa
from files import *
from pascal_voc_writer import Writer
from google.colab.patches import cv2_imshow
from IPython.display import display 

ia.seed(100)

dir = '/content/drive/MyDrive/capstone/augment5/' 
images, annotations = read_train_dataset(dir)


for idx in range(len(images)):
    image = images[idx]
    boxes = annotations[idx][0]

    ia_bounding_boxes = []
    for box in boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

    seq = iaa.Sequential([
        iaa.Multiply((0.5, 1.5)),
        iaa.Affine(
            translate_px={"x": (-30,30), "y": (-30,30)},
            scale=(0.5, 1),
            rotate= (-15, 15)
        ),
        iaa. Fliplr(0.5)
    ])

    seq_det = seq.to_deterministic()

    xml = seq_det.augment_images([image])[0]
    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

    new_image_file = dir + 'aug5_' + annotations[idx][2]
    cv2.imwrite(new_image_file, xml)

    h, w = np.shape(xml)[0:2]
    voc_writer = Writer(new_image_file, w, h)

    for i in range(len(bbs_aug.bounding_boxes)):
        bb_box = bbs_aug.bounding_boxes[i]
        voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

    voc_writer.save(dir + 'aug5_' + annotations[idx][1])
    
    image_before = bbs.draw_on_image(image, thickness=20)
    image_after = bbs_aug.draw_on_image(xml, thickness=20, color=[0, 0, 255])

    # display(image_before)

    #cv2_imshow(cv2.resize(image_before, (380, 640)))
    cv2_imshow(cv2.resize(image_after, (380, 640)))

    # cv2.waitKey(0)

Output hidden; open in https://colab.research.google.com to view.